In [ ]:
# default_exp MNL_from_scratch

# MNL from scratch

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("Choice Net")

'Hello Choice Net!'

In [ ]:
assert say_hello("Choice Net") == "Hello Choice Net!"

##### GPU

In [ ]:
import numpy as np
from numpy.linalg import inv
from numpy.core.umath_tests import inner1d


def initialize_gpu():
    from cudamat import cudamat as cm
    global cm

    # this class wraps matrix operations so that you can switch between numpy
    # and cuda operations
    cm.cuda_set_device(0)
    cm.init()

# the keyword "inplace" for all of these functions re-uses the memory wherever
# possible use this if you don't want allocate more memory on the gpu which can
# be expensive, and don't need to reuse the matrix being transformed


def random(size, typ='numpy'):
    return PMAT(np.random.uniform(size=size).reshape(1, size))

In [ ]:
initialize_gpu

<function __main__.initialize_gpu()>

##### PMAT

In [ ]:
class PMAT:

    def __init__(self, mat, typ='numpy'):
        self.typ = typ
        if (type(mat) != np.ndarray and
                type(mat) != np.matrix and
                type(mat) != np.float64):
            self.typ = 'cuda'
            self.mat = mat
        elif typ == 'numpy':
            self.mat = mat
        elif typ == 'cuda':
            self.mat = cm.CUDAMatrix(mat)

    def multiply(self, mat):
        if self.typ == 'numpy':
            return PMAT(np.dot(self.mat, mat.mat))
        elif self.typ == 'cuda':
            return PMAT(cm.dot(self.mat, mat.mat))

    def exp(self, inplace=False):
        if self.typ == 'numpy':
            return PMAT(np.exp(self.mat))
        elif self.typ == 'cuda':
            if inplace:
                self.mat = cm.exp(self.mat)
                return self
            else:
                target = cm.empty(self.mat.shape)
                cm.exp(self.mat, target=target)
                return PMAT(target)

    def log(self, inplace=False):
        if self.typ == 'numpy':
            return PMAT(np.log(self.mat))
        elif self.typ == 'cuda':
            if inplace:
                self.mat = cm.log(self.mat)
                return self
            else:
                target = cm.empty(self.mat.shape)
                cm.log(self.mat, target=target)
                return PMAT(target)

    # find the first positive value along an axis - for use in doing random
    # choice
    def firstpositive(self, axis):
        if self.typ == 'numpy':
            return PMAT(np.argmax((self.mat + 1.0).astype('i4'), axis=axis))

    def cumsum(self, axis):
        if self.typ == 'numpy':
            return PMAT(np.cumsum(self.mat, axis=axis))
        # elif self.typ == 'cuda':
        #  return PMAT(misc.cumsum(self.mat,axis=axis))

    def max(self, axis):
        if self.typ == 'numpy':
            return PMAT(np.max(self.mat, axis=axis))
        elif self.typ == 'cuda':
            return PMAT(self.mat.max(axis=axis))

    def argmax(self, axis):
        if self.typ == 'numpy':
            return PMAT(np.argmax(self.mat, axis=axis))

    def transpose(self):
        if self.typ == 'numpy':
            return PMAT(np.transpose(self.mat))
        # WARNING, always in place
        elif self.typ == 'cuda':
            self.mat.transpose()

    def reshape(self, rowlen, collen):
        if rowlen == -1:
            rowlen = self.size() // collen
        if collen == -1:
            collen = self.size() // rowlen
        if self.typ == 'numpy':
            self.mat = np.reshape(self.mat, (rowlen, collen), order='F')
            return self
        # WARNING, always in place
        elif self.typ == 'cuda':
            self.mat = self.mat.reshape((rowlen, collen))
            return self

    def size(self):
        if self.typ == 'numpy':
            return self.mat.size
        elif self.typ == 'cuda':
            return self.mat.shape[0] * self.mat.shape[1]

    def sum(self, axis, shorten=0):
        if self.typ == 'numpy':
            # this is weird, but a numpy sum return flat array sometimes and
            # we want 2D matrices
            # return PMAT(np.sum(self.mat,axis=axis,dtype="float64"))
            if axis == 0:
                return PMAT(np.reshape(
                    np.sum(self.mat, axis=axis, dtype="float64"), (1, -1)))
            if axis == 1:
                return PMAT(np.reshape(
                    np.sum(self.mat, axis=axis, dtype="float64"), (-1, 1)))
            # return
            # PMAT(np.array(np.matrix(self.mat).sum(axis=axis,dtype="float64")))
        elif self.typ == 'cuda':
            return PMAT(self.mat.sum(axis=axis))

    def get_mat(self):
        if self.typ == 'numpy':
            return self.mat
        if self.typ == 'cuda':
            return self.mat.asarray()

    def shape(self):
        return self.mat.shape

    def subtract(self, mat, inplace=False):
        if self.typ == 'numpy':
            return PMAT(self.mat - mat.mat)
        if self.typ == 'cuda':
            if inplace:
                self.mat.subtract(mat.mat)
                return self
            else:
                target = cm.empty(self.mat.shape)
                self.mat.subtract(mat.mat, target=target)
                return PMAT(target)

    def divide_by_row(self, rowvec, inplace=False):
        if self.typ == 'numpy':
            return PMAT(self.mat / rowvec.mat)
        elif self.typ == 'cuda':
            if inplace:
                rowvec.mat.reciprocal()
                self.mat.mult_by_row(rowvec.mat)
                return self
            else:
                target = cm.empty(self.mat.shape)
                rowvec.mat.reciprocal()
                self.mat.mult_by_row(rowvec.mat, target=target)
                return PMAT(target)

    def multiply_by_row(self, rowvec, inplace=False):
        if self.typ == 'numpy':
            return PMAT(self.mat * rowvec.mat)
        elif self.typ == 'cuda':
            if inplace:
                self.mat.mult_by_row(rowvec.mat)
                return self
            else:
                target = cm.empty(self.mat.shape)
                self.mat.mult_by_row(rowvec.mat, target=target)
                return PMAT(target)

    def multiply_by_col(self, colvec, inplace=False):
        if self.typ == 'numpy':
            return PMAT(self.mat * colvec.mat)
        elif self.typ == 'cuda':
            if inplace:
                self.mat.mult_by_col(colvec.mat)
                return self
            else:
                target = cm.empty(self.mat.shape)
                self.mat.mult_by_col(colvec.mat, target=target)
                return PMAT(target)

    def add_row_vec(self, rowvec, inplace=False):
        if self.typ == 'numpy':
            return PMAT(self.mat + rowvec.mat)
        elif self.typ == 'cuda':
            if inplace:
                self.mat.add_row_vec(rowvec.mat)
                return self
            else:
                target = cm.empty(self.mat.shape)
                self.mat.add_row_vec(rowvec.mat, target=target)
                return PMAT(target)

    def add_col_vec(self, colvec, inplace=False):
        if self.typ == 'numpy':
            return PMAT(self.mat + colvec.mat)
        elif self.typ == 'cuda':
            if inplace:
                self.mat.add_col_vec(colvec.mat)
                return self
            else:
                target = cm.empty(self.mat.shape)
                self.mat.add_col_vec(colvec.mat, target=target)
                return PMAT(target)

    def element_multiply(self, mat, inplace=False):
        if self.typ == 'numpy':
            return PMAT(self.mat * mat.mat)
        elif self.typ == 'cuda':
            if inplace:
                self.mat = self.mat.mult(mat.mat)
                return self
            else:
                target = cm.empty(self.mat.shape)
                self.mat.mult(mat.mat, target=target)
                return PMAT(target)

    def element_add(self, mat, inplace=False):
        if self.typ == 'numpy':
            return PMAT(self.mat + mat.mat)
        elif self.typ == 'cuda':
            if inplace:
                self.mat = self.mat.add(mat.mat)
                return self
            else:
                target = cm.empty(self.mat.shape)
                self.mat.add(mat.mat, target=target)
                return PMAT(target)

    def clamptomin(self, val):
        assert self.typ == "numpy"
        self.mat[self.mat < val] = val

    def inftoval(self, val):
        assert self.typ == "numpy"
        self.mat[np.isinf(self.mat)] = val

    def nantoval(self, val):
        assert self.typ == "numpy"
        self.mat[np.isnan(self.mat)] = val

    def __str__(self):
        if self.typ == 'numpy':
            return str(self.mat)
        elif self.typ == 'cuda':
            return str(self.get_mat())


##### MNL

In [ ]:
import logging

import numpy as np
import pandas as pd
import scipy.optimize

# from . import pmat
# from .pmat import PMAT

# from ..utils.logutil import log_start_finish

logger = logging.getLogger(__name__)

# right now MNL can only estimate location choice models, where every equation
# is the same
# it might be better to use stats models for a non-location choice problem

# data should be column matrix of dimensions NUMVARS x (NUMALTS*NUMOBVS)
# beta is a row vector of dimensions 1 X NUMVARS


def mnl_probs(data, beta, numalts):
    logging.debug('start: calculate MNL probabilities')
    clamp = data.typ == 'numpy'
    utilities = beta.multiply(data)
    if numalts == 0:
        raise Exception("Number of alternatives is zero")
    utilities.reshape(numalts, utilities.size() // numalts)

    # https://stats.stackexchange.com/questions/304758/softmax-overflow
    utilities = utilities.subtract(utilities.max(0))

    exponentiated_utility = utilities.exp(inplace=True)
    if clamp:
        exponentiated_utility.inftoval(1e20)
    if clamp:
        exponentiated_utility.clamptomin(1e-300)
    sum_exponentiated_utility = exponentiated_utility.sum(axis=0)
    probs = exponentiated_utility.divide_by_row(
        sum_exponentiated_utility, inplace=True)
    if clamp:
        probs.nantoval(1e-300)
    if clamp:
        probs.clamptomin(1e-300)

    logging.debug('finish: calculate MNL probabilities')
    return probs


def get_hessian(derivative):
    return np.linalg.inv(np.dot(derivative, np.transpose(derivative)))


def get_standard_error(hessian):
    return np.sqrt(np.diagonal(hessian))

# data should be column matrix of dimensions NUMVARS x (NUMALTS*NUMOBVS)
# beta is a row vector of dimensions 1 X NUMVARS


def mnl_loglik(beta, data, chosen, numalts, weights=None, lcgrad=False,
               stderr=0):
    logger.debug('start: calculate MNL log-likelihood')
    numvars = beta.size
    numobs = data.size() // numvars // numalts

    beta = np.reshape(beta, (1, beta.size))
    beta = PMAT(beta, data.typ)

    probs = mnl_probs(data, beta, numalts)

    # lcgrad is the special gradient for the latent class membership model
    if lcgrad:
        assert weights
        gradmat = weights.subtract(probs).reshape(probs.size(), 1)
        gradarr = data.multiply(gradmat)
    else:
        if not weights:
            gradmat = chosen.subtract(probs).reshape(probs.size(), 1)
        else:
            gradmat = chosen.subtract(probs).multiply_by_row(
                weights).reshape(probs.size(), 1)
        gradarr = data.multiply(gradmat)

    if stderr:
        gradmat = data.multiply_by_row(gradmat.reshape(1, gradmat.size()))
        gradmat.reshape(numvars, numalts * numobs)
        return get_standard_error(get_hessian(gradmat.get_mat()))

    chosen.reshape(numalts, numobs)
    if weights is not None:
        if probs.shape() == weights.shape():
            loglik = ((probs.log(inplace=True)
                       .element_multiply(weights, inplace=True)
                       .element_multiply(chosen, inplace=True))
                      .sum(axis=1).sum(axis=0))
        else:
            loglik = ((probs.log(inplace=True)
                       .multiply_by_row(weights, inplace=True)
                       .element_multiply(chosen, inplace=True))
                      .sum(axis=1).sum(axis=0))
    else:
        loglik = (probs.log(inplace=True).element_multiply(
            chosen, inplace=True)).sum(axis=1).sum(axis=0)

    if loglik.typ == 'numpy':
        loglik, gradarr = loglik.get_mat(), gradarr.get_mat().flatten()
    else:
        loglik = loglik.get_mat()[0, 0]
        gradarr = np.reshape(gradarr.get_mat(), (1, gradarr.size()))[0]

    logger.debug('finish: calculate MNL log-likelihood')
    return -1 * loglik, -1 * gradarr

In [ ]:
def mnl_simulate(data, coeff, numalts, GPU=False, returnprobs=True):
    """
    Get the probabilities for each chooser choosing between `numalts`
    alternatives.

    Parameters
    ----------
    data : 2D array
        The data are expected to be in "long" form where each row is for
        one alternative. Alternatives are in groups of `numalts` rows per
        choosers. Alternatives must be in the same order for each chooser.
    coeff : 1D array
        The model coefficients corresponding to each column in `data`.
    numalts : int
        The number of alternatives available to each chooser.
    GPU : bool, optional
    returnprobs : bool, optional
        If True, return the probabilities for each chooser/alternative instead
        of actual choices.

    Returns
    -------
    probs or choices: 2D array
        If `returnprobs` is True the probabilities are a 2D array with a
        row for each chooser and columns for each alternative.

    """
    logger.debug(
        'start: MNL simulation with len(data)={} and numalts={}'.format(
            len(data), numalts))
    atype = 'numpy' if not GPU else 'cuda'

    data = np.transpose(data)
    coeff = np.reshape(np.array(coeff), (1, len(coeff)))

    data, coeff = PMAT(data, atype), PMAT(coeff, atype)

    probs = mnl_probs(data, coeff, numalts)

    if returnprobs:
        return np.transpose(probs.get_mat())

    # convert to cpu from here on - gpu doesn't currently support these ops
    if probs.typ == 'cuda':
        probs = PMAT(probs.get_mat())

    probs = probs.cumsum(axis=0)
    r = pmat.random(probs.size() // numalts)
    choices = probs.subtract(r, inplace=True).firstpositive(axis=0)

    logger.debug('finish: MNL simulation')
    return choices.get_mat()


In [ ]:
def mnl_estimate(data, chosen, numalts, GPU=False, coeffrange=(-3, 3),
                 weights=None, lcgrad=False, beta=None):
    """
    Calculate coefficients of the MNL model.

    Parameters
    ----------
    data : 2D array
        The data are expected to be in "long" form where each row is for
        one alternative. Alternatives are in groups of `numalts` rows per
        choosers. Alternatives must be in the same order for each chooser.
    chosen : 2D array
        This boolean array has a row for each chooser and a column for each
        alternative. The column ordering for alternatives is expected to be
        the same as their row ordering in the `data` array.
        A one (True) indicates which alternative each chooser has chosen.
    numalts : int
        The number of alternatives.
    GPU : bool, optional
    coeffrange : tuple of floats, optional
        Limits of (min, max) to which coefficients are clipped.
    weights : ndarray, optional
    lcgrad : bool, optional
    beta : 1D array, optional
        Any initial guess for the coefficients.

    Returns
    -------
    log_likelihood : dict
        Dictionary of log-likelihood values describing the quality of
        the model fit.
    fit_parameters : pandas.DataFrame
        Table of fit parameters with columns 'Coefficient', 'Std. Error',
        'T-Score'. Each row corresponds to a column in `data` and are given
        in the same order as in `data`.

    See Also
    --------
    scipy.optimize.fmin_l_bfgs_b : The optimization routine used.

    """
    logger.debug(
        'start: MNL fit with len(data)={} and numalts={}'.format(
            len(data), numalts))
    atype = 'numpy' if not GPU else 'cuda'

    numvars = data.shape[1]
    numobs = data.shape[0] // numalts

    if chosen is None:
        chosen = np.ones((numobs, numalts))  # used for latent classes

    data = np.transpose(data)
    chosen = np.transpose(chosen)

    data, chosen = PMAT(data, atype), PMAT(chosen, atype)
    if weights is not None:
        weights = PMAT(np.transpose(weights), atype)

    if beta is None:
        beta = np.zeros(numvars)
    bounds = [coeffrange] * numvars

    with log_start_finish('scipy optimization for MNL fit', logger):
        args = (data, chosen, numalts, weights, lcgrad)
        bfgs_result = scipy.optimize.fmin_l_bfgs_b(mnl_loglik,
                                                   beta,
                                                   args=args,
                                                   fprime=None,
                                                   factr=10,
                                                   approx_grad=False,
                                                   bounds=bounds
                                                   )

    if bfgs_result[2]['warnflag'] > 0:
        logger.warn("mnl did not converge correctly: %s",  bfgs_result)

    beta = bfgs_result[0]
    stderr = mnl_loglik(
        beta, data, chosen, numalts, weights, stderr=1, lcgrad=lcgrad)

    l0beta = np.zeros(numvars)
    l0 = -1 * mnl_loglik(l0beta, *args)[0]
    l1 = -1 * mnl_loglik(beta, *args)[0]

    log_likelihood = {
        'null': float(l0[0][0]),
        'convergence': float(l1[0][0]),
        'ratio': float((1 - (l1 / l0))[0][0])
    }

    fit_parameters = pd.DataFrame({
        'Coefficient': beta,
        'Std. Error': stderr,
        'T-Score': beta / stderr})

    logger.debug('finish: MNL fit')
    return log_likelihood, fit_parameters

##### TESTING

In [ ]:
# Check data
tvmodes = pd.read_csv('./data/travel_mode.csv')
tvchoice = pd.read_csv('./data/travel_choosers.csv')

In [ ]:
tvmodes.shape, tvchoice.shape

((840, 10), (8, 10))

In [ ]:
tvmodes.head()

,Unnamed: 0,individual,mode,choice,wait,vcost,travel,gcost,income,size
0,1.air,1,air,False,69,59,100,70,35,1
1,1.train,1,train,False,34,31,372,71,35,1
2,1.bus,1,bus,False,35,25,417,70,35,1
3,1.car,1,car,True,0,10,180,30,35,1
4,2.air,2,air,False,64,58,68,68,30,2


In [ ]:
tvchoice

,Unnamed: 0,individual,mode,choice,wait,vcost,travel,gcost,income,size
0,107.air,107,air,False,69,108,180,128,35,1
1,107.train,107,train,False,34,89,901,187,35,1
2,107.bus,107,bus,False,35,44,891,141,35,1
3,107.car,107,car,True,0,34,720,112,35,1
4,182.air,182,air,False,69,59,121,72,26,1
5,182.train,182,train,False,34,31,386,73,26,1
6,182.bus,182,bus,False,35,25,431,72,26,1
7,182.car,182,car,True,0,14,270,43,26,1


In [ ]:
dm = tvmodes
# chosen = 
# num_alts = 

SyntaxError: invalid syntax (<ipython-input-31-4ec9bba80912>, line 2)

In [ ]:
log_like, fit = mnl_estimate(dm.values, chosen, num_alts)

In [ ]:
def test_mnl_estimate(dm, chosen, num_alts, test_data):
    log_like, fit = mnl.mnl_estimate(dm.values, chosen, num_alts)
    result = pd.Series(fit.Coefficient.values, index=dm.columns)
    result, expected = result.align(test_data['est_expected'])
    npt.assert_allclose(result.values, expected.values, rtol=1e-4)

In [ ]:
columns=['air', 'train', 'bus', 'car']

In [ ]:
tvmodes.head()

,Unnamed: 0,individual,mode,choice,wait,vcost,travel,gcost,income,size
0,1.air,1,air,False,69,59,100,70,35,1
1,1.train,1,train,False,34,31,372,71,35,1
2,1.bus,1,bus,False,35,25,417,70,35,1
3,1.car,1,car,True,0,10,180,30,35,1
4,2.air,2,air,False,64,58,68,68,30,2


In [ ]:
tvmodes[columns]

KeyError: "None of [Index(['air', 'train', 'bus', 'car'], dtype='object')] are in the [columns]"